In [1]:
import yfinance as yf
import re
import pandas as pd
from pandas import DataFrame, Series

In [2]:
%pylab inline
from datetime import datetime

Populating the interactive namespace from numpy and matplotlib


In [3]:
TSM = yf.Ticker('TSM')

In [4]:
df = TSM.history(period = '2y', interval = '60m')

In [5]:
df.shape

(3532, 7)

In [6]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2019-09-03 11:30:00-04:00,41.724998,41.810001,41.650002,41.654999,0,0.0,0
2019-09-03 12:30:00-04:00,41.650002,41.669998,41.599998,41.615002,324204,0.0,0
2019-09-03 13:30:00-04:00,41.615002,41.680000,41.575001,41.680000,307858,0.0,0
2019-09-03 14:30:00-04:00,41.669998,41.790001,41.630001,41.755001,431795,0.0,0
2019-09-03 15:30:00-04:00,41.755001,41.849998,41.700001,41.709999,758889,0.0,0
...,...,...,...,...,...,...,...
2021-09-02 15:30:00-04:00,120.930000,120.967903,120.709999,120.820000,973878,0.0,0
2021-09-03 09:30:00-04:00,121.580002,124.120003,121.510002,123.570000,3912820,0.0,0
2021-09-03 10:30:00-04:00,123.570000,124.239998,123.260002,124.201302,1364008,0.0,0


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [8]:
RNN_df = df[['Close', 'Volume']]

In [9]:
#notes for prediction
seq_len = 150
future_period_predict = 3
ratio_to_predict= 'TSM' 

#classification target
def classify(current, future):
    if float(future)> float(current):
        return 1
    else:
        return 0
    
#shift timeseries down 3 spots for 3 hour prediction
three_hour_future = RNN_df['Close'].shift(-future_period_predict)

#create a 3 hour future column
RNN_df["3 hour future"] = three_hour_future


C:\Users\rudra\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
RNN_df=RNN_df.dropna()

#send close and 3d future to classify function
def grab_arguments(s):
    return classify(s[0],s[1])

#create a classification
RNN_df['target'] = RNN_df[['Close','3 hour future']].T.apply(grab_arguments)


In [11]:

RNN_df

,Close,Volume,3 hour future,target
Datetime,,,,
2019-09-03 11:30:00-04:00,41.654999,0,41.755001,1
2019-09-03 12:30:00-04:00,41.615002,324204,41.709999,1
2019-09-03 13:30:00-04:00,41.680000,307858,42.799999,1
2019-09-03 14:30:00-04:00,41.755001,431795,42.950001,1
2019-09-03 15:30:00-04:00,41.709999,758889,42.994999,1
...,...,...,...,...
2021-09-02 12:30:00-04:00,120.669998,649987,120.820000,1
2021-09-02 13:30:00-04:00,120.540001,496781,123.570000,1
2021-09-02 14:30:00-04:00,120.925003,779426,124.201302,1


In [12]:
################################
##split into train and test set
#############################

#make sure index is sorted. probs already was
times = sorted(RNN_df.index)

# last 5% of time as test set
last_10pct = times[-int(0.1*len(times))]
print(last_10pct)


2021-06-23 15:30:00-04:00


In [13]:
# test set is last 10% and beyond
test_set = RNN_df[RNN_df.index >= last_10pct]
test_set
#train set is anything under the last 10%
train_set =RNN_df[(RNN_df.index < last_10pct)]
train_set

,Close,Volume,3 hour future,target
Datetime,,,,
2019-09-03 11:30:00-04:00,41.654999,0,41.755001,1
2019-09-03 12:30:00-04:00,41.615002,324204,41.709999,1
2019-09-03 13:30:00-04:00,41.680000,307858,42.799999,1
2019-09-03 14:30:00-04:00,41.755001,431795,42.950001,1
2019-09-03 15:30:00-04:00,41.709999,758889,42.994999,1
...,...,...,...,...
2021-06-23 10:30:00-04:00,116.320000,1353712,116.400002,1
2021-06-23 11:30:00-04:00,116.754997,705316,116.500000,0
2021-06-23 12:30:00-04:00,116.320000,581401,116.040001,0


In [14]:
#Drop the right answer, 3 hour future
train_set = train_set.T.drop(['3 hour future']).T
test_set = test_set.T.drop(['3 hour future']).T
len(train_set)
len(test_set)

352

In [15]:
#test_set

In [16]:
## check for infinte values in your data

#train_set['Volume'].preprocessing.scale(train_set['Volume'])
import numpy as np
#check for inf as well as -inf
#mask1 = isinf(-train_set['Volume'])
#train_set[mask1]


#mask = isinf(test_set['Volume'])
#test_set[mask]

# mask = train_set['Volume'] == float(inf)
# train_set[mask]= 0

train_set.loc[(train_set['Volume'] == float("inf")),'Volume']=0
train_set[train_set["Volume"] == 0]

# Replacing infinite with nan
#train_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#train_set.dropna(inplace=True)
  
# Printing df


# Replacing infinite with nan
#test_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#test_set.dropna(inplace=True)
  
# Printing df
#len(test_set)
#len(train_set)
#find the infinite values to get rid of
# 2021-05-05 13:30:00-04:00
#train_set.T["2021-05-05 13:30:00-04:00"]
#drop that index or indices
#RNN_df=RNN_df[~RNN_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#len(RNN_df)

,Close,Volume,target
Datetime,,,
2019-09-03 11:30:00-04:00,41.654999,0.0,1.0
2019-11-29 13:00:00-05:00,53.067402,0.0,0.0
2019-12-24 13:00:00-05:00,57.724602,0.0,1.0
2020-05-14 11:30:00-04:00,50.509998,0.0,1.0
2020-11-27 13:00:00-05:00,98.739998,0.0,0.0
2020-12-24 13:00:00-05:00,105.970001,0.0,1.0
2021-05-05 12:30:00-04:00,116.251900,0.0,0.0


In [17]:
mask = (test_set['Volume'] == float("inf"))
test_set[mask]

,Close,Volume,target
Datetime,,,


In [18]:
#train_set = train_set.drop('2019-12-02 09:30:00-05:00')
#train_set = train_set.drop('2019-12-26 09:30:00-05:00')
#train_set = train_set.drop('2020-05-14 12:30:00-04:00')
#train_set = train_set.drop('2020-12-28 09:30:00-05:00')
#train_set = train_set.drop('2021-05-05 14:30:00-04:00')
#train_set = train_set.drop('2020-11-30 09:30:00-05:00')

In [19]:
###################
#Preprocessing - scale and normalize
#####################

#import preprocessing
from sklearn import preprocessing

# import deque continual appending to list and maintains list len
from collections import deque

import numpy as np
import random

def preprocess_df(df):
    df.dropna(inplace=True)
    
    for col in df:
        
        if col != "target":
            #normalize with % change 
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            #scale the data
            df.loc[(df['Volume'] == float("inf")),'Volume']=0
            #print(df[df[col] == float("inf")])
            df[col]= preprocessing.scale(df[col].values)
        
    
    df.dropna(inplace = True)
    
    sequential_data = []
    #maintain the seq_len
    prev_days = deque(maxlen=seq_len)
    #turns df into a list of lists
    for i in df.values:
        # take each value except for target col in list
        prev_days.append([n for n in i[:-1]])
        #print(len(prev_days))
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days), i[-1]])
            #print(sequential_data)
    
    random.shuffle(sequential_data)  
    
    #####################
    ### Balancet the Data 
    ######################

    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target ==1:
            buys.append([seq,target])
     
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells)) 
    buys = buys[:lower]
    sells = sells[:lower]
    #print(sells)
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), y 

#preprocess_df(train_set)
train_x, train_y = preprocess_df(train_set)
test_x, test_y = preprocess_df(test_set)

In [20]:
print(f"train data:{len(train_x)} validation: {len(test_x)}")

train data:2944 validation: 194


In [21]:
print(f"Dont buys:{train_y.count(0)}, buys:{train_y.count(1)}")

Dont buys:1472, buys:1472


In [22]:
print(f"VALIDATION Dont buys: {test_y.count(0)}, buys: {test_y.count(1)}")

VALIDATION Dont buys: 97, buys: 97


In [33]:

epochs = 3
batch_size = 64

#name = f"{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}"


In [24]:
# check to make sure training and test set are numpy arrays
test_y = np.asarray(test_y)
train_y = np.asarray(train_y)

In [43]:
def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(LSTM(128, input_shape= (train_x.shape[1:])))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(tf.keras.layers.Flatten())
    #dense layer activation rectified linear
    model.add(Dense(32, activation ='relu'))
    model.add(Dropout(0.2))

    #output layer
    model.add(Dense(2, activation = 'softmax'))
    return model


def train(lr, decay, opt_name):
    model = build_model()
    if opt_name.lower() == "sgd":
        opt = tf.keras.optimizers.SGD(lr= lr, decay=decay)
    else:
        opt = tf.keras.optimizers.Adam(lr= lr, decay=decay)
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer = opt,
                 metrics= ['accuracy'])
    #tensorboard = TensorBoard(log_dir = f'log/{name}')
    
    name = f"lr--{lr}--decay--{decay}-opt-{opt_name}"

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = f'log/{name}')

    #checkpoint = ModelCheckpoint(filepath= 'models/',verbose=1, save_best_only=True)

    history = model.fit(
                train_x, train_y,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(test_x, test_y),
                callbacks=[tensorboard_callback])
#model.save('')

In [45]:
train(.005, 1e-4, "adam")

Epoch 1/3
46/46 [==============================] - 85s 2s/step - loss: 0.9136 - accuracy: 0.5107 - val_loss: 0.6962 - val_accuracy: 0.5155
Epoch 2/3
46/46 [==============================] - 73s 2s/step - loss: 0.7292 - accuracy: 0.5014 - val_loss: 0.6929 - val_accuracy: 0.4845
Epoch 3/3
46/46 [==============================] - 71s 2s/step - loss: 0.6986 - accuracy: 0.5082 - val_loss: 0.6940 - val_accuracy: 0.5052


In [30]:
#DELETE LOGS


Invalid switch - "*".
